In [3]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import uproot
from math import pi



In [4]:
filepath = '/eos/uscms/store/user/cmsdas/2025/short_exercises/muons/'
filenames= ['DYJets.root', 'TTToLL.root', 'QCDMuEnriched.root']
variables= ['pt', 'eta', 'isGlobal', 'isTracker', 'nStations', 'nTrackerLayers', 'dxy', 'pfRelIso04_all', 'tkRelIso']
muonid= ['looseId', 'mediumId', 'tightId', 'softId', 'highPtId']
muonflavs= ['prompt', 'tau', 'lf', 'fake']
bins=[20, 25, 2, 2, 6, 25, 40, 30, 30]
binranges=[[0,100], [-2.5, 2.5], [-0.5, 1.5], [-0.5, 1.5], [-0.5, 5.5], [-0.5, 24.5], [-0.10, 0.11], [0, 0.3], [0, 0.3]]

index=[0,1,2]
n=len(filenames)
file=[None]*n
muon=[None]*n
muon_genpartflav=[None]*n
npvs=[None]*n
npvsgood=[None]*n

eventsDY = NanoEventsFactory.from_root(
    filepath + filenames[0],
    treepath = "Events",
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
    entry_stop = 100000,
).events()

eventsTT = NanoEventsFactory.from_root(
    filepath + filenames[1],
    treepath = "Events",
    schemaclass=NanoAODSchema,
    metadata={"dataset": "TT"},
    entry_stop = 100000,    
).events()


eventsQCD = NanoEventsFactory.from_root(
    filepath + filenames[2],
    treepath = "Events",
    schemaclass=NanoAODSchema,
    metadata={"dataset": "QCD"},
    entry_stop = 100000,    
).events()

events = [eventsDY, eventsTT, eventsQCD]

#getting the primary vertices
for idx in index:
    npvsgood[idx]= events[idx].PV.npvsGood
    events[idx]["Muon","PV_npvs"] = events[idx].PV.npvs
    print (events[idx].Muon.PV_npvs)
    muon[idx] = events[idx].Muon



FileNotFoundError: file not found

    '/eos/uscms/store/user/cmsdas/2025/short_exercises/muons/DYJets.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in str/bytes and pathlib.Path.
         Examples: Path("rel/*.root"), "/abs/*.root:tdirectory/ttree"
   * dict: keys are filesystem paths, values are objects-within-ROOT paths.
         Example: {"/data_v1/*

In [5]:
diclist=[]
for idx in index:
    muon_mask=(muon[idx].pt>20)& (abs(muon[idx].eta)<2.4)
    dic={}
    for flav, type in zip([1, 15 , 3, 0], muonflavs):
        muon_mask_all= muon_mask& (muon[idx].genPartFlav==flav)
        event_mask= npvsgood[idx]>0 #at least one good primary vertex
        for var in variables:
            dic[f'{var}_{type}']=(muon[idx])[f'{var}'][muon_mask_all][event_mask]
        for id in muonid:        
            id_mask_combined=muon_mask_all& (((muon[idx])[f'{id}'])!=0)
            dic[f'{id}_{type}_pt']=(muon[idx])['pt'][id_mask_combined][event_mask]
    
    muon_mask_tightid=muon_mask& (muon[idx].tightId!=0)
    muon_mask_tightid_pfreliso04_all=muon_mask_tightid& (muon[idx].pfRelIso04_all<0.15)
    dic['npvs_muon_mask']= (muon[idx].PV_npvs)[event_mask]
    dic['npvs_muon_mask_tightid']=(muon[idx].PV_npvs)[muon_mask_tightid][event_mask]
    dic['npvs_muon_mask_tightid_pfreliso04_all']=(muon[idx].PV_npvs)[muon_mask_tightid_pfreliso04_all][event_mask]
    diclist.append(dic)

combined_variables={}
for type in muonflavs:
        for var in variables:
            combined_variables[f'{var}_{type}']=np.concatenate((ak.flatten(diclist[0][f'{var}_{type}']), ak.flatten(diclist[1][f'{var}_{type}']), ak.flatten(diclist[2][f'{var}_{type}'])))
        for id in muonid:
            combined_variables[f'{id}_{type}_pt']=np.concatenate((ak.flatten(diclist[0][f'{id}_{type}_pt']), ak.flatten(diclist[1][f'{id}_{type}_pt']), ak.flatten(diclist[2][f'{id}_{type}_pt'])))
for name in ['npvs_muon_mask','npvs_muon_mask_tightid','npvs_muon_mask_tightid_pfreliso04_all']:
    combined_variables[name]=np.concatenate((ak.flatten(diclist[0][name]), ak.flatten(diclist[1][name]), ak.flatten(diclist[2][name])))

for var, bin, binrange in zip(variables, bins, binranges):
    plt.figure(figsize=(9,6))
    for type, color in zip(muonflavs, ['red', 'blue', 'green', 'black']):
        plt.hist((combined_variables[f'{var}_{type}']), bins=bin, range=binrange, density=True, histtype='step', linewidth=2, color=f'{color}', label=f'{var}_{type}')
    plt.legend()
    plt.xlabel(f'{var}')
    plt.ylabel('Events')
    #plt.yscale('log')       



AttributeError: 'NoneType' object has no attribute 'pt'